# Working with psydat files

In [ ]:
from psychopy.misc import fromFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import

A psydat file can be imported using the psychopy `fromFile` function: 

In [ ]:
psydata = fromFile("example.psydat")

### Contents

This returns a Python object that contains all of the trial conditions and results as attributes.
These attributes can be listed using the Python `vars` function:

In [ ]:
for var in vars(psydata):
    print(var, end=" | ")

### Conditions

The trial conditions are in `trialList`, each element in this list is a dict of trial conditions that defines a trial:

In [ ]:
psydata.trialList

This can be more easily viewed if converted to a pandas DataFrame:

In [ ]:
pd.DataFrame(psydata.trialList)

The `weight` of a trial is how many times it should be repeated.
This information is also stored in the `trialWeights` list, so for example `trialList[i]` will be repeated `trialWeights[i]` times

In [ ]:
psydata.trialWeights

A block consists of doing a trial with each condition in `trialList` `weight` times, where `weight` can be a different number for each trial, so the total number of trials done in a block is then given by

$$
nTrials = \sum_i^{nConditions} weight[i]
$$

This block is then repeared `nReps` times:

In [ ]:
psydata.nReps

The total number of trials `nTotal` is then given by

$$
nTotal = nReps \times \left( \sum_i^{nConditions} weight[i] \right)
$$

In [ ]:
psydata.nTotal

The condition used for a given trial `iTrial` and repetition number `iRep` is given by `sequenceIndices[iTrial][iRep]`, which gives the index of the conditions used in the `trialList` list:

In [ ]:
psydata.sequenceIndices

The `method` specifies the order in which the trials were done:

- sequential
  - the same order as `trialList`
- random
  - order of trials shuffled within each block
- fullRandom
  - order of trials fully shuffled

In [ ]:
psydata.method

### Results

The results of the trials are in `data` which contains a dict of numpy arrays of recorded data:

In [ ]:
for key, value in psydata.data.items():
    print(key, value.shape)

Each of these is a `nTrials` x `nReps` 2d array, where each element of this array contains the results from the corresponding trial for this variable (which might itself be a single value, e.g. `target_pos`, or an array of values, e.g. `timestamps`)

Which set of conditions was used is given by the `sequenceIndices` entry in the same location

### Plot of results for each trial

For example, a scatter plot of the mouse positions for each trial, labelled by the condition, trial number and repetition number:

In [ ]:
colors = ["blue", "green", "red", "cyan", "magenta", "yellow", "black", "orange"]

nTrials, nReps = psydata.sequenceIndices.shape
fig, axs = plt.subplots(nTrials, nReps, figsize=(8, 8))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        target_radius = psydata.trialList[condition]["target_size"]
        central_target_radius = psydata.trialList[condition]["central_target_size"]
        ax = axs[loc]
        ax.set_title(f"Trial {trial}, Rep {rep} [Condition {condition}]")
        for positions, target_pos, color in zip(
            psydata.data["mouse_positions"][loc],
            psydata.data["target_pos"][loc],
            colors,
        ):
            ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    target_pos,
                    target_radius,
                    edgecolor="none",
                    facecolor=color,
                    alpha=0.1,
                )
            )
        if not psydata.trialList[condition]["automove_cursor_to_center"]:
            for positions, color in zip(
                psydata.data["mouse_positions_back"][loc],
                colors,
            ):
                ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    [0, 0],
                    central_target_radius,
                    edgecolor="none",
                    facecolor="black",
                    alpha=0.1,
                )
            )

plt.show()

### Plot of all trials combined for each condition

Here we instead make one plot for each set of conditions in `trialList`, and super-impose all of the corresponding results:

In [ ]:
nConditions = len(psydata.trialList)
nTrials, nReps = psydata.sequenceIndices.shape
fig, axs = plt.subplots(nConditions, 1, figsize=(16, 16 * nConditions))
axs = np.reshape(axs, (nConditions))  # ensure axs is a 1d-array
for trial in range(nTrials):
    for rep in range(nReps):
        loc = (trial, rep)
        condition = psydata.sequenceIndices[loc]
        target_radius = psydata.trialList[condition]["target_size"]
        ax = axs[condition]
        ax.set_title(f"Condition {condition}")
        for positions, target_pos, color in zip(
            psydata.data["mouse_positions"][loc],
            psydata.data["target_pos"][loc],
            colors,
        ):
            ax.plot(positions[:, 0], positions[:, 1], color=color)
            ax.add_patch(
                plt.Circle(
                    target_pos,
                    target_radius,
                    edgecolor="none",
                    facecolor=color,
                    alpha=0.1,
                )
            )
plt.show()